In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

## ODDS API

In [ ]:
# {
#     "key": "basketball_ncaab",
#     "group": "Basketball",
#     "title": "NCAAB",
#     "description": "US College Basketball",
#     "active": true,
#     "has_outrights": false
#   },

In [ ]:
from_str = datetime(2024, 3, 9, 4, 15).strftime('%Y-%m-%dT%H:%M:%SZ')  # 2024-03-09T00%3A00%3A00Z
to_str = datetime(2024, 4, 20, 4, 15).strftime('%Y-%m-%dT%H:%M:%SZ')  # 2024-04-20T00%3A00%3A00Z

In [ ]:
spreads_upcoming = requests.get(
    f'https://api.the-odds-api.com/v4/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=decimal&commenceTimeFrom={from_str}&commenceTimeTo={to_str}'
).json()


In [ ]:
len(spreads_upcoming)

In [ ]:
spreads_upcoming[0].keys()

In [ ]:
date_str = datetime(2024, 3, 23, 15, 15).strftime('%Y-%m-%dT%H:%M:%SZ') # 2024-03-23T12%3A15%3A00Z
spreads_past = requests.get(
    f'https://api.the-odds-api.com/v4/historical/sports/basketball_ncaab/odds?apiKey={config["ODDS_API_KEY"]}&regions=us&markets=spreads&dateFormat=iso&oddsFormat=american&date={date_str}'
).json()

In [ ]:
len(spreads_past['data'])

In [ ]:
spreads_past['data'][0].keys()

## ESPN API

In [ ]:
data_from_march = requests.get(
    'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20240322-20240324'
).json()

In [ ]:
data_322 = requests.get(
    'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20240322'
).json()
events_322 = set([event['shortName'] for event in data_322['events']])
len(events_322)

In [ ]:
data_323 = requests.get(
    'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20240328'
).json()
events_323 = set([event['shortName'] for event in data_323['events']])
len(events_323)

In [ ]:
data_323.keys()#['keys']

In [ ]:
len(events_322 | events_323)

In [ ]:
data_3224 = requests.get(
    'https://site.api.espn.com/apis/site/v2/sports/basketball/mens-college-basketball/scoreboard?dates=20240322-20240324'
).json()
events_3224 = set([event['shortName'] for event in data_3224['events']])
len(events_3224)

In [ ]:
events_323 - events_3224

In [ ]:
len(data_from_march['events'])

In [ ]:
short_teams, long_teams = [], []
for event in data_from_march['events']:
    s_teams = event['shortName'].split(' VS ')
    assert len(s_teams) == 2
    l_teams = event['name'].split(' at ')
    assert len(l_teams) == 2
    short_teams += s_teams
    long_teams += l_teams

long_to_short = dict(zip(long_teams, short_teams))


In [ ]:
short_to_long = dict(zip(short_teams, long_teams))

In [ ]:
event_types = set()
for event in data_from_march['events']:
    event_types.add(event['season']['slug'])

In [ ]:
test_df = pd.read_csv('monkey-madness-teams.csv')

In [ ]:
def normalize_team_name(team_name):
    n = team_name.strip(".").strip()
    if n.endswith("'s"):
        return n[:-2]
    return n

In [ ]:
def team_name_to_short_code(team_name_norm):
    short_team_code = None
    for long_team, short_team in long_to_short.items():
        if team_name_norm.lower() in long_team.lower() or long_team.lower() in team_name_norm.lower():
            short_team_code = short_team
        elif short_team.lower() in team_name_norm.lower():
            short_team_code = short_team
    return short_team_code

In [ ]:
test_df['team_name_norm'] = test_df.team_name.apply(normalize_team_name)
test_df['team_code'] = test_df.team_name_norm.apply(team_name_to_short_code)

In [ ]:
# fix empty and duplicates

In [ ]:
code_counts = dict(test_df.team_code.value_counts())

In [ ]:
dup_codes = [key for key, value in code_counts.items() if value > 1]

In [ ]:
dup_codes

In [ ]:
no_code = test_df[pd.isna(test_df.team_code)]
print(len(no_code))
no_code.head(15)

In [ ]:
dup_code = test_df[test_df.team_code.apply(lambda c: c in dup_codes)]
print(len(dup_code))
dup_code.head(15)

In [ ]:
fixes = {
    'Molloy (2)': 'FLA',
    'Nicolai': 'MORE',
    'F-Cup': 'LBSU',
    'Scott (2)': 'DUKE',
    'Cimi (2)': 'UK',
    'Peralo (2)': 'CSU',
    'Leff (2)': 'TEX'
}

In [ ]:
test_df['team_code'] = test_df.apply(
    lambda row: fixes[row.participant_name] if row.participant_name in fixes else row.team_code,
    axis=1
)

In [ ]:
test_df.isna().sum()

In [ ]:
len(test_df.team_code.unique())

In [ ]:
test_df.team_code.value_counts()

In [ ]:
test_df[test_df.team_code.values == 'TEX'].head()

In [ ]:
test_df[test_df.team_code.values == 'TTU'].head()

In [ ]:
len(test_df.team_code.unique())

In [ ]:
test_df.to_csv('mm-with-team-codes.csv', index=False)

In [ ]:
# manually changed to TTU^

In [ ]:
df_clean = pd.read_csv('mm-with-team-codes.csv')
assert len(df_clean.team_code.unique()) == 64

In [ ]:
from construct_bracket import Bracket

In [ ]:
bracket = Bracket.from_csv('mm-with-team-codes.csv')

In [ ]:
bracket

In [ ]:
data_from_march.keys()

In [ ]:
is_finished = data_from_march['events'][0]['status']['type']['completed']  # boolean
status = data_from_march['events'][0]['status']['type']['name']
# change last 0 to 1 for second team
team_to_score = {}
for i in range(2):
    score = data_from_march['events'][0]['competitions'][0]['competitors'][i]['score']
    team_code = data_from_march['events'][0]['competitions'][0]['competitors'][i]['team']['abbreviation']
    team_to_score[team_code] = score
status, is_finished, team_to_score

In [ ]:
data_from_march['events'][0]['competitions'][0]['competitors'][0]['winner']